<a href="https://colab.research.google.com/github/Tech-nawaf111/Flutter-Projects/blob/main/breed_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.applications import xception
from keras.models import Model
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import os
import glob

In [ ]:
csv_path = '/content/drive/MyDrive/dataset/breeds_details.csv'
breed_path = '/content/drive/MyDrive/dataset/realbreeds'


In [ ]:
def train_test_split(file_path):
  #read All paths and labels in the csv
  train_set = pd.read_csv(csv_path)
  #group the labels into array against label like bread1 = [image_path1, image_path2,...]
  train_set = train_set.groupby(['name']).image_path.apply(lambda x: list(x.values)).reset_index()
  #from each array take 4 images and add it to another pandas named test 
  #in the same manner label
  test_set = pd.DataFrame(data={'name': [], 'image_path': []})
  for index, row in train_set.iterrows():
    test_paths = []
    paths = row['image_path']
    name = row['name']
    test_paths.extend(paths[-40:-1])
    del paths[-40:-1]
    test_set = test_set.append({'name': name,'image_path': test_paths},ignore_index=True)
    train_set.at[41,'image_path'] = paths
  train_set.dropna(subset = ["name"], inplace=True)
  test_set.dropna(subset = ["name"], inplace=True)
  return train_set,test_set

In [ ]:

# For Triplet Loss, the objective is to build triplets
# <anchor, positive, negative> consisting of an anchor 
# image, a positive image (which is similar to the anchor image)
# , and a negative image (which is dissimilar to the anchor image). 
# There are different ways to define similar and dissimilar images.
#  If you have a dataset having multiple labels as the target class,
#   Then images of the same class can be considered as similar,
#    and images across different classes can be considered as dissimilar.

def generate_triplets(names, image_paths): #Ids, classes:
  triplets = []
  num_labels = len(names) 
  for index in range(num_labels):
      # set the current name as the positive label
      positive_label = names[index] 
      # select a random negative label and select a negative random image from there
      negative_index = np.random.randint(0, len(image_paths))
      while index == negative_index:
          negative_index = np.random.randint(0, len(image_paths))
      negative_label = names[negative_index]
      # make sure that negative image label is not the same as anchor,positive
      while positive_label == negative_label:
          negative_label = np.random.choice(names)
      # for the list against the name for each image select a positive random image from the list 
      images = image_paths[index]
      for index2 in range(len(images)):  
          # set the current image as anchor
          anchor = images[index2]
          #select a random image to be the positive from the selected class
          postive = images[np.random.randint(0, len(images))]
          # make sure that anchor and positive are the same image
          while anchor == postive:
              postive = images[np.random.randint(0, len(images))]
          # select a image at random to be the negative class from the negative label images   
          negative = image_paths[negative_index][np.random.randint(0, len(image_paths[negative_index]))]
          triplets.append([anchor,postive, negative,positive_label,negative_label])
  return triplets

In [ ]:
df1,df2 = train_test_split(file_path = csv_path)
print(df2)
train_dataset = generate_triplets(df1['name'].to_list(),df1['image_path'].to_list(),)
test_dataset = generate_triplets(df2['name'].to_list(),df2['image_path'].to_list(),)
print(train_dataset)
print(test_dataset)
del df1,df2

     name                                         image_path
0  breed1  [breed1_223, breed1_224, breed1_225, breed1_22...
1  breed2  [breed2_198, breed2_199, breed2_200, breed2_20...
2  breed3  [breed3_214, breed3_215, breed3_216, breed3_21...
3  breed4  [breed4_212, breed4_213, breed4_214, breed4_21...
4  breed5  [breed5_164, breed5_165, breed5_166, breed5_16...
[['breed1_1', 'breed1_152', 'breed5_124', 'breed1', 'breed5'], ['breed1_2', 'breed1_91', 'breed5_8', 'breed1', 'breed5'], ['breed1_3', 'breed1_158', 'breed5_50', 'breed1', 'breed5'], ['breed1_4', 'breed1_103', 'breed5_81', 'breed1', 'breed5'], ['breed1_5', 'breed1_154', 'breed5_150', 'breed1', 'breed5'], ['breed1_6', 'breed1_61', 'breed5_136', 'breed1', 'breed5'], ['breed1_7', 'breed1_155', 'breed5_87', 'breed1', 'breed5'], ['breed1_8', 'breed1_169', 'breed5_2', 'breed1', 'breed5'], ['breed1_9', 'breed1_72', 'breed5_82', 'breed1', 'breed5'], ['breed1_10', 'breed1_125', 'breed5_10', 'breed1', 'breed5'], ['breed1_11', 'breed1_13

In [ ]:
print(len(train_dataset),len(test_dataset))



1009 195


In [ ]:
from keras import backend as K
def triplet_loss(anchor, positive, negative, alpha = 0.4):   

    # Compute the distance between the anchor and the positive
    distance_positive = tf.math.reduce_sum(tf.math.pow( (anchor - positive) , tf.constant(2.0)), axis = 1)
    # Compute the distance between the anchor and the negative
    distance_negative = tf.math.reduce_sum(tf.math.pow( (anchor - negative) , tf.constant(2.0)), axis = 1)
    # compute loss
    basic_loss = distance_positive - distance_negative + alpha
    loss = K.maximum(basic_loss,0.0)

    return tf.reduce_mean(loss)

In [ ]:
input = keras.Input(shape=(299, 299, 3))

xception_model = xception.Xception(include_top=False,input_shape=(299,299,3) ,weights="imagenet")
pretrained_model = Model(inputs = xception_model.input, outputs = xception_model.get_layer("block14_sepconv2_act").output)
for layer in pretrained_model.layers:
    layer.trainable = False

output = pretrained_model(input)

model = Model(input, output)
#optimization algorithm for stochastic gradient descent for training deep learning models. 
optimizer = tf.keras.optimizers.Adam(0.00001)
model.compile(loss=triplet_loss, optimizer= optimizer, metrics=[tf.keras.metrics.Accuracy()])

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 model_2 (Functional)        (None, 10, 10, 2048)      20861480  
                                                                 
Total params: 20,861,480
Trainable params: 0
Non-trainable params: 20,861,480
_________________________________________________________________


In [ ]:
def readImage(name, folder, target_size = (299,299,3)):
  path = breed_path + '/' + folder + '/' + name;
  print(path)
  image = tf.keras.preprocessing.image.load_img(path)
  image_arr = tf.keras.preprocessing.image.img_to_array(image)
  image_arr = tf.image.resize(image_arr,(target_size[0], target_size[1])).numpy()
  #return np.array(image_arr/255.)
  return np.expand_dims(image_arr/255., axis= 0)

In [ ]:
import time
epochs = 10
loss_value = 0
for epoch in range(epochs):
  for triplet in train_dataset:
    anchor = readImage(triplet[0]+str('.png'),triplet[3])
    positive = readImage(triplet[1]+str('.png'),triplet[3])
    negative = readImage(triplet[2]+str('.png'),triplet[4])
    with tf.GradientTape() as tape:
        anchor,positive,negative = model(anchor, training=True), model(positive, training=True), model(negative, training=True)
        loss_value = triplet_loss(anchor,positive,negative)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    del anchor,positive,negative
print('Loss:{}'.format(loss_value))


Streaming output truncated to the last 5000 lines.
/content/drive/MyDrive/dataset/realbreeds/breed5/breed5_121.png
/content/drive/MyDrive/dataset/realbreeds/breed2/breed2_131.png
/content/drive/MyDrive/dataset/realbreeds/breed2/breed2_100.png
/content/drive/MyDrive/dataset/realbreeds/breed5/breed5_152.png
/content/drive/MyDrive/dataset/realbreeds/breed2/breed2_132.png
/content/drive/MyDrive/dataset/realbreeds/breed2/breed2_169.png
/content/drive/MyDrive/dataset/realbreeds/breed5/breed5_121.png
/content/drive/MyDrive/dataset/realbreeds/breed2/breed2_133.png
/content/drive/MyDrive/dataset/realbreeds/breed2/breed2_72.png
/content/drive/MyDrive/dataset/realbreeds/breed5/breed5_106.png
/content/drive/MyDrive/dataset/realbreeds/breed2/breed2_134.png
/content/drive/MyDrive/dataset/realbreeds/breed2/breed2_114.png
/content/drive/MyDrive/dataset/realbreeds/breed5/breed5_73.png
/content/drive/MyDrive/dataset/realbreeds/breed2/breed2_135.png
/content/drive/MyDrive/dataset/realbreeds/breed2/breed2

In [ ]:
model.save_weights('content/drive/My Drive/FYP_ML/')


# Model Testing

In [ ]:
model.load_weights('content/drive/My Drive/FYP_ML/')
model.save("model.h5")

In [ ]:
error = 0
total = len(test_dataset)
for triplet in test_dataset:
  a, p, n = readImage(triplet[0]+str('.png'),triplet[3]),readImage(triplet[1]+str('.png'),triplet[3]),readImage(triplet[2]+str('.png'),triplet[4])
  print(triplet)

  anchor, positive, negative = model(a), model(p), model(n)
  dist_anchor_positive = tf.math.reduce_sum(tf.math.pow( (anchor - positive) , tf.constant(2.0)), axis = 1)
  dist_anchor_negative = tf.math.reduce_sum(tf.math.pow( (anchor - negative) , tf.constant(2.0)), axis = 1)

  dist_anchor_positive = tf.reduce_mean(dist_anchor_positive).numpy()
  dist_anchor_negative = tf.reduce_mean(dist_anchor_negative).numpy()

  if (dist_anchor_positive > dist_anchor_negative):
    error += 1

/content/drive/MyDrive/dataset/realbreeds/breed1/breed1_223.png
/content/drive/MyDrive/dataset/realbreeds/breed1/breed1_250.png
/content/drive/MyDrive/dataset/realbreeds/breed2/breed2_235.png
['breed1_223', 'breed1_250', 'breed2_235', 'breed1', 'breed2']
/content/drive/MyDrive/dataset/realbreeds/breed1/breed1_224.png
/content/drive/MyDrive/dataset/realbreeds/breed1/breed1_258.png
/content/drive/MyDrive/dataset/realbreeds/breed2/breed2_235.png
['breed1_224', 'breed1_258', 'breed2_235', 'breed1', 'breed2']
/content/drive/MyDrive/dataset/realbreeds/breed1/breed1_225.png
/content/drive/MyDrive/dataset/realbreeds/breed1/breed1_259.png
/content/drive/MyDrive/dataset/realbreeds/breed2/breed2_233.png
['breed1_225', 'breed1_259', 'breed2_233', 'breed1', 'breed2']
/content/drive/MyDrive/dataset/realbreeds/breed1/breed1_226.png
/content/drive/MyDrive/dataset/realbreeds/breed1/breed1_254.png
/content/drive/MyDrive/dataset/realbreeds/breed2/breed2_199.png
['breed1_226', 'breed1_254', 'breed2_199', 

In [ ]:
triplets = [['breed3_18.png', 'breed1_2.png', 'breed2_17.png', 'breed3', 'breed1','breed2'],
            ['breed3_18.png', 'breed3_5.png', 'breed4_22.png', 'breed3', 'breed3','breed4'],
            ['breed3_18.png', 'breed5_11.png', 'breed6_9.png', 'breed3', 'breed5','breed6'],
            ['breed3_18.png', 'breed2_17.png', 'breed1_2.png', 'breed3', 'breed2','breed1'],
            ['breed3_18.png', 'breed4_22.png', 'breed3_5.png', 'breed3', 'breed4','breed3'],
            ['breed3_18.png', 'breed6_9.png', 'breed5_11.png', 'breed3', 'breed6','breed5']]



for triplet in triplets:
  anchor, positive, negative = model(readImage(triplet[0],triplet[3])), model(readImage(triplet[1],triplet[4])), model(readImage(triplet[2],triplet[5]))
  dist_anchor_positive = tf.math.reduce_sum(tf.math.pow( (anchor - positive) , tf.constant(2.0)), axis = 1)
  dist_anchor_negative = tf.math.reduce_sum(tf.math.pow( (anchor - negative) , tf.constant(2.0)), axis = 1)
  dist_anchor_positive = tf.reduce_mean(dist_anchor_positive).numpy()
  dist_anchor_negative = tf.reduce_mean(dist_anchor_negative).numpy()
  
  if (dist_anchor_positive > dist_anchor_negative):
    print('error')
  else:
      print(dist_anchor_positive)

# New Section

In [ ]:
print(error)
print(total)
print('Accuracy: ' , 1 - error/total)

67
195
Accuracy:  0.6564102564102564


In [ ]:
#model.predict(readImage(triplet[2],triplet[4]))
print(model(readImage(triplet[0],triplet[3]),readImage(triplet[1],triplet[3]),readImage(triplet[2],triplet[4])))

In [ ]:
import tensorflow as tf

# Convert the model
saved_model_dir = 'content/drive/My Drive/FYP_ML_model/'
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)